In [1]:
##############################
# Configuration & base setup #
##############################

from config import load_config
from adapters import BlizzardAuctionHouseClient, ParquetWriter
from usecases import FetchAndStoreAuctionsUseCase

# Load configuration
CONFIG = load_config()

print(f"Region: {CONFIG.blizzard.api_region.value}")
print(f"Storage directory: {CONFIG.storage.local.root_directory}")

Region: eu
Storage directory: /Users/julien.leloup/Personal/ShadowDrive/wow-analytics-data


In [2]:
##############################
# Fetch and store auction data
##############################

# Create adapters (infrastructure layer)
writer = ParquetWriter(
    root_dir=CONFIG.storage.local.root_directory,
    region=CONFIG.blizzard.api_region.value,
    compression="snappy",
)

async def main():
    # Create the API client adapter (uses async context manager)
    async with BlizzardAuctionHouseClient(
        client_id=CONFIG.blizzard.api_client_id,
        client_secret=CONFIG.blizzard.api_client_secret.get_secret_value(),
        region=CONFIG.blizzard.api_region.value,
        locale=CONFIG.blizzard.api_locale,
    ) as client:
        # Create use case with injected dependencies
        use_case = FetchAndStoreAuctionsUseCase(
            auction_house=client,
            storage=writer,
        )
        
        # Execute the use case - fetch and store realm metadata
        df_realms = await use_case.fetch_and_store_realm_metadata()

        await use_case.fetch_and_store_realm_auctions()
        
        print(f"\nStored {len(df_realms)} realms to {writer.data_dir}")

# Run
await main()

Successfully authenticated. Token expires in 86399 seconds

Fetching connected realms...
Found 92 connected realms

Fetching realm details...
Saved 92 connected realms to /Users/julien.leloup/Personal/ShadowDrive/wow-analytics-data/eu/global/connected_realms.parquet

Fetching connected realm IDs...
Found 92 connected realms

Fetching auctions for 92 realms...
Fetching auctions for realm 1080...
Fetching auctions for realm 1084...
Fetching auctions for realm 1596...
Fetching auctions for realm 1597...
Fetching auctions for realm 1598...
Completed realm 1080: 12386 auctions
Fetching auctions for realm 1602...
Completed realm 1598: 18576 auctions
Fetching auctions for realm 1604...
Completed realm 1596: 23564 auctions
Fetching auctions for realm 1605...
Completed realm 1597: 40545 auctions
Fetching auctions for realm 1614...
Completed realm 1605: 4298 auctions
Completed realm 1604: 16344 auctions
Fetching auctions for realm 604...
Fetching auctions for realm 1091...
Completed realm 1084: 